In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

# Send an HTTP request to the URL
url = "https://harpalssokhi.com"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find all links inside div with class "col-md-12 images-blog"
  # div_1_links = [a['href'] for div in soup.find_all('div', class_='col-md-12 item-img-box') for a in div.find_all('a')]
  categories = [[a.string, a['href']] for div in soup.find_all('div', id='most-rated02') for h3 in div.find_all('h3') for a in h3.find_all('a')]
  # Print the links
  print("Links in div class '':")
  for category in categories:
      print(category)
  categories.append(['Indian', 'https://harpalssokhi.com/cuisine/indian/'])
  print('Total Cuisine types:', len(categories))
else:
  print("Failed to retrieve the web page.")

Links in div class '':
['Punjabi', 'https://harpalssokhi.com/cuisine/punjabi/']
['Maharashtra', 'https://harpalssokhi.com/cuisine/maharashtra/']
['International', 'https://harpalssokhi.com/cuisine/international/']
['Gujarat', 'https://harpalssokhi.com/cuisine/gujarati/']
['South Indian', 'https://harpalssokhi.com/cuisine/south-indian/']
['Uttar Pradesh', 'https://harpalssokhi.com/cuisine/uttar-pradesh/']
['Rajasthan', 'https://harpalssokhi.com/cuisine/rajasthan/']
['Bengal', 'https://harpalssokhi.com/cuisine/bengal/']
['Madhya Pradesh', 'https://harpalssokhi.com/cuisine/madhya-pradesh/']
Total Cuisine types: 10


In [ ]:
data = []
for category in categories:
  res = requests.get(category[1], headers=headers)
  if res.status_code == 200:
    soup = BeautifulSoup(res.content, 'html.parser')
    child_pages = [int(a.string) for div in soup.find_all('div', class_='navigation') for a in div.find_all('a') if a.string.isnumeric() ]
    tot_pages = 1 if len(child_pages) == 0 else max(child_pages)

    for page in range(1, tot_pages + 1):
      url = f'{category[1]}/page/{page}/'
      res = requests.get(url, headers=headers)
      if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        data += [
            {'dish_name': a.string,'dish_link': a['href'], 'category' : category[0]}
            for div in soup.find_all('div',class_='container-recipe')
            for h3 in div.find_all('h3')
            for a in h3.find_all('a')]
      else:
        print("Failed to retrieve the child page.")
  else:
    print("Failed to retrieve the web page.")

In [ ]:
print(len(data))

1253


In [ ]:
data

[{'dish_name': 'Rajma Palak Methi',
  'dish_link': 'https://harpalssokhi.com/recipe/rajma-palak-methi-2/',
  'category': 'Punjabi'},
 {'dish_name': 'Vegetable Shahi Korma.',
  'dish_link': 'https://harpalssokhi.com/recipe/vegetable-shahi-korma/',
  'category': 'Punjabi'},
 {'dish_name': 'Veg Kathi Roll.',
  'dish_link': 'https://harpalssokhi.com/recipe/veg-kathi-roll/',
  'category': 'Punjabi'},
 {'dish_name': 'Patiala Chicken',
  'dish_link': 'https://harpalssokhi.com/recipe/patiala-chicken/',
  'category': 'Punjabi'},
 {'dish_name': 'Paneer Angoori',
  'dish_link': 'https://harpalssokhi.com/recipe/paneer-angoori/',
  'category': 'Punjabi'},
 {'dish_name': 'Ginger Mint Lassi',
  'dish_link': 'https://harpalssokhi.com/recipe/ginger-mint-lassi/',
  'category': 'Punjabi'},
 {'dish_name': 'Baingan Masala',
  'dish_link': 'https://harpalssokhi.com/recipe/baingan-masala/',
  'category': 'Punjabi'},
 {'dish_name': 'Veg Angara.',
  'dish_link': 'https://harpalssokhi.com/recipe/veg-angara/',
 

In [ ]:
for d in data:
  res = requests.get(d['dish_link'], headers=headers)
  if res.status_code == 200:
    soup = BeautifulSoup(res.content, 'html.parser')
    extracted_text = [li.get_text(strip=True) for div in soup.find_all('div', class_='recipe-meta') for li in div.find_all('li') ]
    if len(extracted_text) == 0: continue
    d['prep_time'] = extracted_text[0].replace('Prep Time','')
    d['cook_time'] = extracted_text[1].replace('Cook Time','')
    d['serves'] = extracted_text[2].replace('Serving','')
    d['difficulty'] = extracted_text[3].replace('Difficulty','')

    d['instructions'] = ""
    recipe_section = soup.find('div', class_='printable')
    if recipe_section:
        paragraphs = recipe_section.find_all('p')
        for para in paragraphs:
          d['instructions'] += para.get_text().replace('\xa0','') + "\n"
    print(d)
  else:
    print("Failed to retrieve the web page.")

{'dish_name': 'Vegetable Shahi Korma.', 'dish_link': 'https://harpalssokhi.com/recipe/vegetable-shahi-korma/', 'category': 'Punjabi', 'prep_time': '20-25 minutes  minutes', 'cook_time': '30 minutes minutes', 'serves': 'For 2-3 portions  People', 'difficulty': 'Easy', 'instructions': 'Ingredients             Quantity\nGhee                  2tbsp\nCinnamon stick          ½ no.\nGreen cardamom        6-7 no.\nCloves                 4no.\nWhole black pepper      ½ tsp\nCumin seeds           1tsp\nOnion chopped          3no.\nGinger garlic paste       1 ½ Tbsp\nTomatoes chopped       2no.\nSalt to taste\nCoriander powder       1 ½ Tbsp\nRed chili powder        1tsp\nTurmeric powder        1tsp\nGaram masala          1tsp\nGreen chilies silt        2-3 no. + 1-2 no.\nWater as required+ as required+ as required\nCurd                 2Tbsp\nCashew nut paste       1Tbsp\nButter                1Tbsp+ for garnish\nGinger juliennes        1tsp+ for garnish\nFresh cream           ½ cup+ for garnish

In [ ]:
data[1].keys()

dict_keys(['dish_name', 'dish_link', 'category', 'prep_time', 'cook_time', 'serves', 'difficulty', 'instructions'])

In [ ]:
### Creating DB ###

import sqlite3

# Create a new SQLite database or connect to an existing one
conn = sqlite3.connect("recipes.db")

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Define the schema with the specified fields
schema = """
CREATE TABLE IF NOT EXISTS recipes_url (
    dish_name TEXT,
    category TEXT,
    prep_time TEXT,
    cook_time TEXT,
    serves TEXT,
    difficulty TEXT,
    instructions TEXT,
    url TEXT
);
"""

# Execute the schema creation query
cursor.execute(schema)

# Commit the changes and close the database connection
conn.commit()
conn.close()

print("SQLite database with schema created successfully.")

SQLite database with schema created successfully.


In [ ]:
conn = sqlite3.connect("recipes.db")
cursor = conn.cursor()

for dish in data:
  if sorted(dish.keys()) != sorted(['dish_name', 'dish_link', 'category', 'prep_time', 'cook_time', 'serves', 'difficulty', 'instructions']): continue
  insert_query = "INSERT INTO recipes_url (dish_name, category, prep_time, cook_time, serves, difficulty, instructions, url) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
  values = (str(dish['dish_name']), str(dish['category']), str(dish['prep_time']), str(dish['cook_time']), str(dish['serves']), str(dish['difficulty']), str(dish['instructions']), str(dish['dish_link']))
  cursor.execute(insert_query, values)
  conn.commit()

In [ ]:
import sqlite3

# Replace 'your_database.db' with the actual name of your SQLite database file.
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM recipes_url")
rows = cursor.fetchall()

for row in rows:
    print(row)
import csv

# Replace 'output.csv' with the desired output file name.
with open('output.csv', 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    cursor.execute("SELECT * FROM recipes_url")
    rows = cursor.fetchall()
    csv_writer.writerows(rows)
conn.close()


('Vegetable Shahi Korma.', 'Punjabi', '20-25 minutes  minutes', '30 minutes minutes', 'For 2-3 portions  People', 'Easy', 'Ingredients             Quantity\nGhee                  2tbsp\nCinnamon stick          ½ no.\nGreen cardamom        6-7 no.\nCloves                 4no.\nWhole black pepper      ½ tsp\nCumin seeds           1tsp\nOnion chopped          3no.\nGinger garlic paste       1 ½ Tbsp\nTomatoes chopped       2no.\nSalt to taste\nCoriander powder       1 ½ Tbsp\nRed chili powder        1tsp\nTurmeric powder        1tsp\nGaram masala          1tsp\nGreen chilies silt        2-3 no. + 1-2 no.\nWater as required+ as required+ as required\nCurd                 2Tbsp\nCashew nut paste       1Tbsp\nButter                1Tbsp+ for garnish\nGinger juliennes        1tsp+ for garnish\nFresh cream           ½ cup+ for garnish\nCarrot cubes           ½ cup\nFrench beans chopped   ½ cup\nGreen peas            ½ cup\nCauliflower florets       ½ cup\nKhoya                2Tbsp\nPaneer cub

In [ ]:
import sqlite3
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Connect to the SQLite database
conn = sqlite3.connect('/content/recipes.db')  # Replace 'your_database.db' with your database path

# Query the database and load data into a Pandas DataFrame
query = "SELECT dish_name, instructions FROM recipes_url"
df = pd.read_sql_query(query, conn)

# Define the dish you want to find similar dishes for
target_dish = "Makai Roti"  # Replace with the dish name you want

# Create a CountVectorizer to convert ingredients into a bag of words
count_vectorizer = CountVectorizer()
ingredient_matrix = count_vectorizer.fit_transform(df['instructions'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(ingredient_matrix)

# Get the index of the target dish
target_index = df[df['dish_name'] == target_dish].index[0]

# Calculate similarity scores for all dishes
similarity_scores = list(enumerate(cosine_sim[target_index]))

# Sort the dishes based on similarity
sorted_similar_dishes = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

# Get the top 2 similar dishes
top_similar_dishes = sorted_similar_dishes[1:4]  # Exclude the target dish itself

# Extract and print the names of the top similar dishes
print("Similarity based on the ingredients used (top 3)")
print("")
for dish_index, similarity_score in top_similar_dishes:
    if similarity_score > 0.9:
          print(f"Dish: {df['dish_name'][dish_index]}, Probably the same dish")
    elif similarity_score > 0.6:
          print(f"Dish: {df['dish_name'][dish_index]}, Similarity: {similarity_score:.2f}")

# Close the database connection
conn.close()


Similarity based on the ingredients used (top 3)

Dish: Makai Roti, Probably the same dish
Dish: Makki di Roti, Similarity: 0.74
Dish: Makki di Roti, Similarity: 0.74


In [ ]:
!pip install transformers

     ---------------------------------------- 8.3/8.3 MB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 2.6 MB/s eta 0:00:00
     -------------------------------------- 269.6/269.6 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 277.8/277.8 kB 1.3 MB/s eta 0:00:00
     ------------------------------------ 152.8/152.8 kB 762.1 kB/s eta 0:00:00
     -------------------------------------- 311.7/311.7 kB 1.6 MB/s eta 0:00:00
     -------------------------------------- 169.0/169.0 kB 1.1 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, DistilBertTokenizer, DistilBertForQuestionAnswering
# import sqlite3
import pandas as pd

# # Connect to the SQLite database
# conn = sqlite3.connect("recipes.db")  # Replace 'your_database.db' with your database path

# # Query the database and load data into a Pandas DataFrame
# query = "SELECT dish_name, instructions FROM recipes_url"
# df = pd.read_sql_query(query, conn)
df = pd.read_excel("finally.xlsx")

# Define the question-answering pipeline using DistilBERT
qa_pipeline = pipeline(
    "question-answering",
    model=DistilBertForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad"),
    tokenizer=DistilBertTokenizer.from_pretrained("distilbert-base-cased-distilled-squad"),
)

# Define a function to get an answer based on a question
def get_answer(question, context):
    result = qa_pipeline(question=question, context=context)
    return result["answer"]

# Example usage
target_dish_index = 10  # Replace with the index of the dish you want to use as context
target_dish_name = df["dish_name"][target_dish_index]
context = df["Process"][target_dish_index]

# Sample questions
questions = ["What is the main ingredient?", "How to cook?", "Any tips for making it better?"]

# Print answers for each question
for question in questions:
    answer = get_answer(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print("-" * 50)

# Close the database connection
# conn.close()


Question: What is the main ingredient?
Answer: cornflakes,
--------------------------------------------------
Question: How to cook?
Answer: coat it with the egg white and cornflakes,
--------------------------------------------------
Question: Any tips for making it better?
Answer: coat it with the egg white and cornflakes,
--------------------------------------------------


In [ ]:
import spacy

# Load a pre-trained spaCy model (e.g., en_core_web_sm)
nlp = spacy.load("en_core_web_sm")

# Example function for entity relation extraction
def extract_entities_relations(text):
    doc = nlp(text)

    # Extract entities and their relationships based on your use case
    cuisines = [ent.text for ent in doc.ents if ent.label_ == "CUISINE"]
    ingredients = [ent.text for ent in doc.ents if ent.label_ == "INGREDIENT"]

    # Assuming ROOT is the recipe name
    recipe_name = [token.text for token in doc if token.dep_ == "ROOT"]

    return recipe_name, cuisines, ingredients


In [ ]:
# Example function for question-answering using spaCy entities
def get_answer(question, context):
    # Use the entity relation extraction model to preprocess the text and extract entities
    recipe_name, cuisines, ingredients = extract_entities_relations(context)

    # Incorporate entities into the context
    enriched_context = f"{context} {' '.join(recipe_name)} {' '.join(cuisines)} {' '.join(ingredients)}"

    # Use a question-answering model for generating answers
    qa_pipeline = pipeline(
        "question-answering",
        model=DistilBertForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad"),
        tokenizer=DistilBertTokenizer.from_pretrained("distilbert-base-cased-distilled-squad"),
    )

    # Get the answer using the question-answering model
    result = qa_pipeline(question=question, context=enriched_context)
    return result["answer"]


In [ ]:
# Example usage
target_dish_index = 15  # Replace with the index of the dish you want to use as context
target_dish_name = df["dish_name"][target_dish_index]
context = df["Process"][target_dish_index]

# Sample questions
questions = ["What is the main ingredient?", "How to cook?", "Any tips for making it better?"]

# Print answers for each question
for question in questions:
    answer = get_answer(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print("-" * 50)


Question: What is the main ingredient?
Answer: cumin seeds, cloves, cardamom,
--------------------------------------------------
Question: How to cook?
Answer: Add water cover and cook for 2-3 whistles.Let
--------------------------------------------------
Question: Any tips for making it better?
Answer: Add water cover and cook for 2-3 whistles.Let
--------------------------------------------------


In [ ]:
#puran poli
#egg pulao
#chapati noodles
#punjabi rajma
#medu vada
#chilli paneer
#raj kachori
#vegetable fried rice

# Example usage
target_dish_name = "vegetable fried rice"  # Replace with the name of the dish you want to use as context

# Find the corresponding index for the specified dish name
target_dish_index = df.index[df["dish_name"] == target_dish_name].tolist()

# Check if the dish name exists in the DataFrame
if target_dish_index:
    target_dish_index = target_dish_index[0]  # Take the first index if there are multiple matches
    context = df["Process"][target_dish_index]

    # Sample questions
    questions = ["What is the main ingredient?", "How to cook?", "Any tips for making it better?"]

    # Print answers for each question
    for question in questions:
        answer = get_answer(question, context)
        print(f"Question: {question}")
        print(f"Answer: {answer}")
        print("-" * 50)
else:
    print(f"Dish with name '{target_dish_name}' not found.")


Question: What is the main ingredient?
Answer: rice
--------------------------------------------------
Question: How to cook?
Answer: minute.Add green chillies,
--------------------------------------------------
Question: Any tips for making it better?
Answer: sugar, soya sauce, white pepper powder, vinegar and salt,
--------------------------------------------------
